In [99]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [100]:
load_dotenv(override=True)

True

In [101]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [102]:
print(linkedin)

   
Contact
sudipto002gayen@gmail.com
www.linkedin.com/in/sudipto-
gayen-7416b622b (LinkedIn)
Top Skills
Back-End Web Development
MongoDB
JavaScript
Certifications
ATG certified Business Development
Associate
Python Essentials 1
Introduction to Cybersecurity
SUDIPTO GAYEN
Passionate about Software Development || Full stack developer
(MERN) || Cyber Security Enthusiastic
Kolkata, West Bengal, India
Summary
I’m a Full-Stack Software Developer passionate about building
secure, scalable, and user-focused web applications. My expertise
lies in the MERN stack (MongoDB, Express.js, React, Node.js)
along with a strong foundation in JavaScript, modern JS libraries/
frameworks, and SQL, as well as NoSQL databases.
I enjoy architecting end-to-end solutions—from designing efficient
database schemas and building REST/GraphQL APIs to crafting
dynamic, responsive frontends. With a growing interest in
cybersecurity, I integrate best practices like OWASP standards,
secure authentication (JWT/OAuth2), a

In [103]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [104]:
name = "Sudipto Gayen"

In [105]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [106]:
system_prompt

"You are acting as Sudipto Gayen. You are answering questions on Sudipto Gayen's website, particularly questions related to Sudipto Gayen's career, background, skills and experience. Your responsibility is to represent Sudipto Gayen for interactions on the website as faithfully as possible. You are given a summary of Sudipto Gayen's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nSudipto Gayen is a MERN stack full-stack developer with a strong foundation in cybersecurity. He has interned at AILABS and DataSpace Academy, and worked at Banao Technologies. Proficient in JavaScript, MongoDB, and back-end development, he studies Computer Science with a cybersecurity focus at The Neotia University.\nYou've interned across tech and business roles, proving you're both creative and strategic. With a passion

In [107]:
google_api_key = os.getenv("GOOGLE_API_KEY")
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"

In [108]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
    return response.choices[0].message.content

In [109]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [110]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [111]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [112]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [113]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [114]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = ollama.beta.chat.completions.parse(model="llama3.2", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [116]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
reply = response.choices[0].message.content

In [117]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response acknowledges the user's question and provides a clear and concise answer. However, it could be more engaging and detailed by elaborating on Sudipto Gayen's experience working on projects that might have led him to learn about patents or intellectual property in software development. Providing additional context or insights would make the response more informative and impressive for potential clients or employers.")

In [118]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
    return response.choices[0].message.content

In [119]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [120]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
